[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mckinsey/vizro/blob/main/vizro-ai/examples/dashboard_by_vizro_ai.ipynb)

In [1]:
# @title
%%html
<style>
  .cell-bg {
    background-color: #E6F3FF;
    padding: 10px;
    width: 100%;
  }
  .disclaimer {
    background-color: #ffffff;
    border: 1px solid #ddd;
    border-radius: 5px;
    padding: 15px;
    margin: 10px 0;
    font-family: Arial, sans-serif;
    color: #000000;
  }
  .disclaimer h3 {
    margin-top: 0;
    color: #000000;
  }
  .disclaimer ol {
    padding-left: 20px;
  }
  .disclaimer li {
    margin-bottom: 10px;
  }
  .disclaimer p {
    font-weight: bold;
    margin-bottom: 0;
  }

  /* Dark mode styles */
  @media (prefers-color-scheme: dark) {
    .cell-bg {
      background-color: #1a1a1a;
    }
    .disclaimer {
      background-color: #2a2a2a;
      border-color: #444;
      color: #ffffff;
    }
    .disclaimer h3 {
      color: #ffffff;
    }
  }
</style>

<div class="cell-bg">
  <div class="disclaimer">
    <h3>Disclaimer: Using Google Colab</h3>
    <p>Please read carefully before proceeding:</p>
    <ol>
      <li><strong>Google's Servers:</strong> This notebook runs on Google's servers. Your code and data will be processed and temporarily stored on Google's infrastructure.</li>
      <li><strong>Data Privacy:</strong> Be mindful of the data you upload or process here. Avoid using sensitive, confidential, or personal information that shouldn't be shared with third parties.</li>
      <li><strong>Security Considerations:</strong> Colab is a shared environment. While Google implements security measures, treat it as a public environment and act accordingly.</li>
      <li><strong>Terms of Service:</strong> Ensure your usage complies with Google Colab's Terms of Service and Google Cloud's Terms of Service.</li>
      <li><strong>Data Persistence:</strong> Data in Colab notebooks is not permanently stored. Save important data and results to your local machine or a secure cloud storage.</li>
      <li><strong>Resource Limitations:</strong> Colab provides free resources, but these are subject to availability and usage limits.</li>
    </ol>
    <p>By using this Colab notebook, you acknowledge that you understand and accept these conditions. Please use Colab at your own discretion and responsibility.</p>
  </div>
</div>

#### Setup the LLM API key

Secure your LLM API key as environment variable
- click the `Secrets` button on the left
- click `Add new secret`
- insert "OPENAI_API_KEY" under `Name`
- insert the API key value under `Value` (use raw string without quote. e.g., `xyz` instead of `"xyz"`)
- [optional] you might also need to insert "OPENAI_BASE_URL" and its value, based on the requirements from your API key provider (use raw string without quote. e.g., `xyz` instead of `"xyz"`)
- click the `Notebook access` toggle to make it available for the notebook run

# Use Vizro-AI to generate Vizro dashboard
# **At the top of this notebook, click `Runtime` -> `Run all`**

In [ ]:
import os

from google.colab import userdata

# @title ## Setup
# @markdown #### 1. Install Vizro-AI
print("Installing dependencies")
%pip install uv -q -q -q
# %pip uninstall ipykernel -y -q -q -q
# %pip install -U ipykernel -q -q -q
!uv pip install --system vizro-ai --quiet
print("✅ Installation finished")

# @markdown #### 2. Setup LLM API access


def _safe_get_userdata(key):
    try:
        return userdata.get(key)
    except Exception as e:
        print(f"Warning: Unable to access {key}. Reason: {e!s}")
        return None


# Always try to set the OPENAI_API_KEY
api_key = _safe_get_userdata("OPENAI_API_KEY")
if api_key:
    os.environ["OPENAI_API_KEY"] = api_key
else:
    print("❌ OPENAI_API_KEY not set. Click `Secrets` icon on the left to setup.")

# Conditionally set OPENAI_BASE_URL if provided and accessible
openai_base_url = _safe_get_userdata("OPENAI_BASE_URL")
if openai_base_url:
    os.environ["OPENAI_BASE_URL"] = openai_base_url
else:
    print("User defined OPENAI_BASE_URL not set. Using default URL.")

print("\nCurrent environment variables:")
print(f"OPENAI_API_KEY: {'✅ Set' if 'OPENAI_API_KEY' in os.environ else 'Not set'}")
print(f"OPENAI_BASE_URL: {'✅ Set' if 'OPENAI_BASE_URL' in os.environ else 'Not set by user, use default'}")

In [ ]:
# @markdown #### Data upload (multiple dataframes accepted)
# @markdown - click `Choose Files` if you'd like to upload and use your own data
# @markdown - or click `Cancel upload` and use the default example data (plotly gapminder) for initial exploration

import pandas as pd
from google.colab import files
from plotly.express.data import gapminder

uploaded = files.upload()

dfs = []

if not uploaded:
    print("No files uploaded. Use the plotly gapminder dataset as an example.")
    df = gapminder()
    dfs.append(df)
else:
    for fn in uploaded.keys():
        print(f'User uploaded file "{fn}"')

        df_uploaded = pd.read_csv(fn)
        dfs.append(df_uploaded)

In [ ]:
from IPython.display import HTML, display
from ipywidgets import Layout, Textarea

# @title ## Input the LLM model choice and user prompt { run: "auto" }

# @markdown #### 1. Choose the LLM model

LLM = "gpt-4o"  # @param ["gpt-4o", "gpt-4-turbo", "gpt-4o-mini"]


# @markdown ---
# @markdown #### 2. Input your dashboard question
# @markdown e.g. "Create a gdp dashboard with 1 page. use a bar chart to show gdp per continent and filter data by country"

default_user_input = """
Create a 2 page dashboard.

<page 1> 1 bar chart, 1 filter
use a bar chart to show the average GDP per continent. Add a filter to filter the bar chart by country.

<page 2> 1 card, 1 table
create a card with text "This is the Gapminder dataset"
use a table to show the data details, showing features like GDP and life expectancy.

page 2 layout: Image the page consists 6 grid units horizontally.
The card takes 1 unit of the page space on the left and the table takes 5 units on the right.
"""
user_input = Textarea(
    value=default_user_input,
    placeholder="Input your requirements for the dashboard",
    description="user_input:",
    disabled=False,
    rows=10,  # This sets the number of visible lines
    layout=Layout(width="auto", height="auto"),  # This makes the width automatic
)
display(user_input)
display(HTML("<br>"))

# @markdown ---


print(f"Selected LLM: {LLM}")
# print(f"User input: {user_input.value}")

In [ ]:
# @title ## Build the dashboard using Vizro-AI

import ipywidgets as widgets
from IPython.display import display


def _run_code(button):
    print("Running VizroAI...")
    import dash._callback_context
    from vizro import Vizro
    from vizro_ai import VizroAI

    dash._callback_context.context_value.set({})
    Vizro._reset()

    vizro_ai = VizroAI(model=LLM)
    res = vizro_ai.dashboard(dfs, user_input.value, return_elements=True)

    vizro_dashboard = Vizro().build(res.dashboard)

    button1.result = vizro_dashboard
    print("✅ Dashboard generated. Check its code and live view below")
    print(res.code)


button1 = widgets.Button(
    description="Build Dashboard!",
    button_style="info",
    tooltip="Click to build the dashboard using Vizro-AI",
)
display(button1)

button1.on_click(_run_code)

In [ ]:
# @title ## Render dashboard

import ipywidgets as widgets
from IPython.display import display


def _run_dashboard(button):
    button1.result.run()


button2 = widgets.Button(
    description="Run Dashboard!",
    button_style="info",
    tooltip="Click to render the Vizro dashboard",
)
display(button2)

button2.on_click(_run_dashboard)